In [1]:

from imblearn.under_sampling import TomekLinks
from imblearn.over_sampling import SMOTE
import pandas as pd
import sklearn
from imblearn.combine import SMOTETomek
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score,confusion_matrix,accuracy_score,classification_report
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,VotingClassifier
from sklearn.preprocessing import PolynomialFeatures,StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
pd.set_option('display.max_columns', 500)
import pickle as pkl

In [2]:
df = pd.read_csv('../polydf.csv',index_col='Unnamed: 0')
df1 = pd.read_csv('../prepeddata.csv',index_col='Unnamed: 0')
df1['stimulant'] = df1['stimulant'].apply(lambda x: 'low' if x <= 2 else 'high')
df
RandomForestClassifier()


RandomForestClassifier()

In [3]:
stimx = df1.drop(['stimulant','hallucinagen','depressant'],axis = 1)
stimy = df1.stimulant
stimx

sm = SMOTE()
tl = TomekLinks()
sampling = SMOTETomek(sampling_strategy='auto', random_state=None, smote=sm, tomek=tl, n_jobs=1)

stimx, stimy = sampling.fit_resample(stimx, stimy)



In [4]:
stimXtrain,stimXtest,stimYtrain,stimYtest = train_test_split(stimx,stimy)

In [5]:
stimscaler = StandardScaler()
stimXtrain = stimscaler.fit_transform(stimXtrain)
stimXtest = stimscaler.transform(stimXtest)
stimYtrain

2930    high
222      low
3041    high
3087    high
2606    high
        ... 
134      low
228      low
586      low
2559    high
171      low
Name: stimulant, Length: 2437, dtype: object

Logistic regression classifiers

In [6]:
# mod = LogisticRegression(class_weight = 'balanced'
#                          ,max_iter=1000,solver='liblinear',warm_start=True,tol=0.2)
# mod.fit(stimXtrain,stimYtrain)
# stimtrianlogypred = mod.predict(stimXtrain)
# stimtestlogypred = mod.predict(stimXtest)

In [7]:
logregparam_grid = {'C':[x/60 for x in list(range(0,60,1))],'max_iter' :[1000],'class_weight':['auto'],'warm_start' :[True,False], 'solver': ['liblinear', 'rbf','sag']}
mod = GridSearchCV(LogisticRegression(), logregparam_grid, n_jobs=-1, cv=5, verbose=1)
mod.fit(stimXtrain,stimYtrain)
stimtrianlogypred = mod.predict(stimXtrain)
stimtestlogypred = mod.predict(stimXtest)


Fitting 5 folds for each of 360 candidates, totalling 1800 fits


C:\Users\Sigma\AppData\Roaming\Python\Python37\site-packages\sklearn\model_selection\_search.py:921: UserWarning: One or more of the test scores are non-finite: [       nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan 0.7529833  0.7529833
        nan        nan        nan        nan 0.75298246 0.75298246
        nan        nan        nan        nan 0.75298415 0.75298415
        nan        nan        nan        nan 0.75257431 0.75257431
        nan        nan        nan        nan 0.75216363 0.75216363
        nan        nan        nan        nan 0.75134396 0.75134396
        nan        nan        nan        nan 0.75134564 0.75134564
        nan        nan        nan        nan 0.75134564 0.75134564
        nan        nan        nan        nan 0.75134564 0.75134564
        nan        nan        nan        nan 0.75134564 0.75134564
        nan        nan        nan        nan 0.75134564 0.75134564
        nan        nan        nan   

In [9]:
polymodel1 = mod.best_estimator_


In [10]:
f1_score(stimYtrain,stimtrianlogypred,average='weighted'),f1_score(stimYtest,stimtestlogypred,average='weighted')

(0.7487518011655647, 0.7396817200054929)

In [11]:
logregparam_grid2 = {'tol': [0,0.0001,0.001,0.1,1, 10]
                     ,'C': [0,0.001,0.1,1, 10]
                     ,'penalty': ['l1','l2','elasticnet']
                     ,'max_iter' :[1000],'class_weight':['auto']
                     ,'warm_start' :[True,False]
                     , 'solver': ['lbfgs','liblinear', 'rbf','sag','elasticnet','saga',]
                     ,'dual':[False,True],"l1_ratio":[None,0.01,0.1,0.2,0.3,0.4]
                    ,'fit_intercept':[True,False]}
mod2 = GridSearchCV(LogisticRegression(), logregparam_grid2, n_jobs=-1, cv=5, verbose=1)
mod2.fit(stimXtrain,stimYtrain)
stimtrianlog2ypred = mod2.predict(stimXtrain)
stimtestlog2ypred = mod2.predict(stimXtest)

Fitting 5 folds for each of 25920 candidates, totalling 129600 fits


C:\Users\Sigma\AppData\Roaming\Python\Python37\site-packages\sklearn\model_selection\_search.py:921: UserWarning: One or more of the test scores are non-finite: [nan nan nan ... nan nan nan]
  category=UserWarning


In [12]:
f1_score(stimYtrain,stimtrianlog2ypred,average='weighted'),f1_score(stimYtest,stimtestlog2ypred,average='weighted')
polymodel2 = mod2.best_estimator_

In [13]:
logregparam_grid3 = {'tol': [0,0.0001,0.001,0.1,1, 10]
                     ,'penalty': ['l1','l2','elasticnet']
                     ,'max_iter' :[4000],'class_weight':['auto']
                     ,'warm_start' :[True]
                     , 'solver': ['lbfgs','liblinear', 'rbf','sag','elasticnet','saga']
                    }
mod3 = GridSearchCV(LogisticRegression(), logregparam_grid3, n_jobs=-1, cv=5, verbose=1)
mod3.fit(stimXtrain,stimYtrain)
stimtrianlog3ypred = mod3.predict(stimXtrain)
stimtestlog3ypred = mod3.predict(stimXtest)

Fitting 5 folds for each of 108 candidates, totalling 540 fits


C:\Users\Sigma\AppData\Roaming\Python\Python37\site-packages\sklearn\model_selection\_search.py:921: UserWarning: One or more of the test scores are non-finite: [       nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
 0.74455852 0.74455852 0.74373717 0.737577   0.73182752 0.7301848
 0.74455852 0.74455852 0.74455852 0.74414784 0.74332649 0.74291581
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
 0.74414784 0.74455852 0.74414784 0.73716632 0.71786448 0.71170431
        nan        nan        nan        nan        nan        nan
 0.74414784 0.74455852 0.74414784 0.73470226 0.72566735 0.72731006
        nan        nan        nan   

In [14]:
f1_score(stimYtrain,stimtrianlog3ypred,average='weighted'),f1_score(stimYtest,stimtestlog3ypred,average='weighted')

(0.7513087818332471, 0.7398855443283363)

In [15]:
polymodel3 = mod3.best_estimator_
#('polymodel1',polymodel1),('polymodel1',polymodel2),('polymodel1',polymodel3)

In [16]:
pkl.dump(polymodel1,open('PolyModel1.pkl','wb'))
pkl.dump(polymodel2,open('PolyModel2.pkl','wb'))
pkl.dump(polymodel3,open('PolyModel3.pkl','wb'))

Decision Trees

In [17]:
# dtc = DecisionTreeClassifier(criterion='gini',ccp_alpha=0.0055)
# dtc.fit(stimXtrain,stimYtrain)
# dtcytrainpred = dtc.predict(stimXtrain)
# dtcytestpred = dtc.predict(stimXtest)
parameters={'max_depth': range(1,20,1),'ccp_alpha':[x/60 for x in list(range(0,60,1))],'max_depth':[4,6,7,9,11],'criterion':['gini']}
dtc = GridSearchCV(DecisionTreeClassifier(),param_grid=parameters,verbose=True,n_jobs=-1)
dtc.fit(stimXtrain,stimYtrain)
dtcytrainpred = dtc.predict(stimXtrain)
dtcytestpred = dtc.predict(stimXtest)

Fitting 5 folds for each of 300 candidates, totalling 1500 fits


In [18]:
f1_score(stimYtrain,dtcytrainpred,average='weighted'),f1_score(stimYtest,dtcytestpred,average='weighted')

(0.9765843229111388, 0.843084895406015)

In [19]:
confusion_matrix(stimYtest,dtcytestpred)

array([[370,  44],
       [ 83, 315]], dtype=int64)

In [20]:
parameters={'criterion':['entropy','gini'],'min_samples_leaf':range(0,20,2),'ccp_alpha':[x/300 for x in list(range(0,60,1))]}
dtc2 = GridSearchCV(DecisionTreeClassifier(),param_grid=parameters,verbose=True,n_jobs=-1)
dtc2.fit(stimXtrain,stimYtrain)
dtcytrainpred = dtc2.predict(stimXtrain)
dtcytestpred = dtc2.predict(stimXtest)

Fitting 5 folds for each of 1200 candidates, totalling 6000 fits


C:\Users\Sigma\AppData\Roaming\Python\Python37\site-packages\sklearn\model_selection\_search.py:921: UserWarning: One or more of the test scores are non-finite: [       nan 0.81848049 0.82874743 ... 0.50184805 0.50184805 0.50184805]
  category=UserWarning


In [21]:
f1_score(stimYtrain,dtcytrainpred,average='weighted'),f1_score(stimYtest,dtcytestpred,average='weighted')

(0.9597505930743382, 0.8594967216147147)

In [22]:
confusion_matrix(stimYtest,dtcytestpred)

array([[365,  49],
       [ 65, 333]], dtype=int64)

In [23]:
# dtc3 = DecisionTreeClassifier(class_weight='balanced',criterion='gini',ccp_alpha=0.006599999)
# dtc3.fit(stimXtrain,stimYtrain)
# dtc3ytrainpred = dtc3.predict(stimXtrain)
# dtc3ytestpred = dtc3.predict(stimXtest)
parameters={'min_impurity_decrease':[0+x/100 for x in range(1,10)],'ccp_alpha':[x/60 for x in list(range(0,100,1))],'criterion':['entropy','gini'],'min_samples_leaf':range(0,20,2)}
dtc3 = GridSearchCV(DecisionTreeClassifier(),param_grid=parameters,verbose=True,n_jobs=-1)
dtc3.fit(stimXtrain,stimYtrain)
dtcytrainpred = dtc3.predict(stimXtrain)
dtcytestpred = dtc3.predict(stimXtest)

Fitting 5 folds for each of 18000 candidates, totalling 90000 fits


C:\Users\Sigma\AppData\Roaming\Python\Python37\site-packages\sklearn\model_selection\_search.py:921: UserWarning: One or more of the test scores are non-finite: [       nan 0.7798768  0.7798768  ... 0.50184805 0.50184805 0.50184805]
  category=UserWarning


In [24]:
f1_score(stimYtrain,dtcytrainpred,average='weighted'),f1_score(stimYtest,dtcytestpred,average='weighted')


(0.7943259846087922, 0.7745211469048117)

In [25]:
polymodel4 = dtc.best_estimator_
polymodel5 = dtc2.best_estimator_
polymodel6 = dtc3.best_estimator_

pkl.dump(polymodel4,open('PolyModel4.pkl','wb'))
pkl.dump(polymodel5,open('PolyModel5.pkl','wb'))
pkl.dump(polymodel6,open('PolyModel6.pkl','wb'))
#('polymodel1',polymodel1),('polymodel1',polymodel2),('polymodel1',polymodel3),('polymodel4',polymodel4),('polymodel5',polymodel5),('polymodel6',polymodel6)


Knearest

In [26]:
parameters={'n_neighbors':range(1,50)}
knn = GridSearchCV(KNeighborsClassifier(),param_grid=parameters,verbose=True,n_jobs=-1)
knn.fit(stimXtrain,stimYtrain)
knnytrainpred = knn.predict(stimXtrain)
knnytestpred = knn.predict(stimXtest)


Fitting 5 folds for each of 49 candidates, totalling 245 fits


In [27]:
parameters={'n_neighbors':range(50,1130)}
knn2 = GridSearchCV(KNeighborsClassifier(),param_grid=parameters,verbose=True,n_jobs=-1)
knn2.fit(stimXtrain,stimYtrain)
knn2ytrainpred = knn2.predict(stimXtrain)
knn2ytestpred = knn2.predict(stimXtest)


Fitting 5 folds for each of 1080 candidates, totalling 5400 fits


In [28]:
polymodel7 = knn.best_estimator_
polymodel8 = knn2.best_estimator_

#('polymodel1',polymodel1),('polymodel1',polymodel2),('polymodel1',polymodel3),('polymodel4',polymodel4),('polymodel5',polymodel5),('polymodel6',polymodel6),('polymodel7',polymodel7),('polymodel8',polymodel8)
pkl.dump(polymodel7,open('PolyModel7.pkl','wb'))
pkl.dump(polymodel8,open('PolyModel8.pkl','wb'))


Random Forest

In [29]:
parameters={'warm_start':[True,False],'class_weight':['balanced'],'criterion':['gini','entropy'],'n_estimators':[20,50,100,500]}
rfc = GridSearchCV(RandomForestClassifier(),param_grid=parameters,verbose=True,n_jobs=-1)
rfc.fit(stimXtrain,stimYtrain)
rfcytrainpred = rfc.predict(stimXtrain)
rfcytestpred = rfc.predict(stimXtest)


Fitting 5 folds for each of 16 candidates, totalling 80 fits


C:\Users\Sigma\AppData\Roaming\Python\Python37\site-packages\sklearn\ensemble\_forest.py:587: UserWarning: class_weight presets "balanced" or "balanced_subsample" are not recommended for warm_start if the fitted data differs from the full dataset. In order to use "balanced" weights, use compute_class_weight ("balanced", classes, y). In place of y you can use a large enough sample of the full training set target to properly estimate the class frequency distributions. Pass the resulting weights as the class_weight parameter.
  warn('class_weight presets "balanced" or '


In [30]:
rfctrainypred = rfc.predict(stimXtrain)
rfctestypred = rfc.predict(stimXtest)
f1_score(rfctrainypred,stimYtrain,average='weighted'),f1_score(rfctestypred,stimYtest,average='weighted')
confusion_matrix(stimYtest,rfctestypred)




array([[393,  21],
       [ 40, 358]], dtype=int64)

In [31]:
parameters={'min_samples_leaf' : [3,5,7],'class_weight':['balanced'],'random_state':[True]
                         ,'criterion':['gini']
                         }
rfc2 = GridSearchCV(RandomForestClassifier(),param_grid=parameters,verbose=True,n_jobs=-1)
rfc2.fit(stimXtrain,stimYtrain)
rfc2ytrainpred = rfc2.predict(stimXtrain)
rfc2ytestpred = rfc2.predict(stimXtest)


Fitting 5 folds for each of 3 candidates, totalling 15 fits


In [32]:
rfc2trainypred = rfc2.predict(stimXtrain)
rfc2testypred = rfc2.predict(stimXtest)
f1_score(rfc2trainypred,stimYtrain,average='weighted'),f1_score(rfc2testypred,stimYtest,average='weighted')



(0.9864476614609666, 0.9065793423798783)

In [33]:
polymodel9 = rfc.best_estimator_
polymodel10 = rfc2.best_estimator_
#('polymodel1',polymodel1),('polymodel1',polymodel2),('polymodel1',polymodel3),('polymodel4',polymodel4),('polymodel5',polymodel5),('polymodel6',polymodel6),('polymodel7',polymodel7),('polymodel8',polymodel8),('polymodel9',polymodel9),('polymodel10',polymodel10)

pkl.dump(polymodel9,open('PolyModel9.pkl','wb'))
pkl.dump(polymodel10,open('PolyModel10.pkl','wb'))

voting classifier

In [34]:

vtc = VotingClassifier([('polymodel1',polymodel1),('polymodel2',polymodel2),('polymodel3',polymodel3)
                        ,('polymodel5',polymodel5),('polymodel6',polymodel6),('polymodel4',polymodel4)
                        ,('polymodel7',polymodel7),('polymodel8',polymodel8),('polymodel9',polymodel9)
                        ,('polymodel10',polymodel10)],voting='soft',verbose = True,n_jobs=-1)
vtc.fit(stimXtrain,stimYtrain)

VotingClassifier(estimators=[('polymodel1',
                              LogisticRegression(C=0.75, class_weight='auto',
                                                 max_iter=1000, solver='sag',
                                                 warm_start=True)),
                             ('polymodel2',
                              LogisticRegression(C=10, class_weight='auto',
                                                 fit_intercept=False,
                                                 max_iter=1000, tol=0,
                                                 warm_start=True)),
                             ('polymodel3',
                              LogisticRegression(class_weight='auto',
                                                 max_iter=4000, penalty='l1',
                                                 solver='saga', tol=0,...
                             ('polymodel7',
                              KNeighborsClassifier(n_neighbors=1)),
                         

In [35]:
vtcypred = vtc.predict(stimXtest)
vtcytrain = vtc.predict(stimXtrain)

print(f1_score(vtcytrain,stimYtrain,average='weighted'),f1_score(stimYtest,vtcypred,average='weighted'))


0.9761856803751263 0.8747216414978388


In [36]:
pkl.dump(vtc,open('vtcpoly.pkl','wb'))

In [37]:
confusion_matrix(vtcytrain,stimYtrain)

array([[1204,   49],
       [   9, 1173]], dtype=int64)

In [38]:
print(classification_report(stimYtrain, vtcytrain))

              precision    recall  f1-score   support

        high       0.96      0.99      0.98      1213
         low       0.99      0.96      0.98      1222

    accuracy                           0.98      2435
   macro avg       0.98      0.98      0.98      2435
weighted avg       0.98      0.98      0.98      2435



In [39]:
print(classification_report(stimYtest, vtcypred))

              precision    recall  f1-score   support

        high       0.83      0.95      0.89       414
         low       0.94      0.80      0.86       398

    accuracy                           0.88       812
   macro avg       0.89      0.87      0.87       812
weighted avg       0.88      0.88      0.87       812

